Vitor Domingos Baldoino dos Santos</br>
Universidade Presbiteriana Mackenzie</br>
Faculdade de Computação e Informática</br>
[vdbaldoino@gmail.com](mailto:vdbaldoino@gmail.com)</br>

Dataset: [Portuguese Tweets for Sentiment Analysis](https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis)

Recursos para problemas que tive:

- Não estava conseguindo implementar tudo que queria apenas utilizando a biblioteca da HuggingFace, então pensei em fazer o fine-tuning apenas com o PyTorch.
    - Para uma abordagem usando apenas o PyTorch, cheguei no link: [BERT Fine-Tuning Tutorial with PyTorch · Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)
    - Para uma abordagem hibrída (HuggingFace + Training Loop do PyTorch), cheguei nessa documentação: [Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training)
    - Publicação de exemplo: [análise de sentimentos em português utilizando Pytorch e Python](https://medium.com/data-hackers/an%C3%A1lise-de-sentimentos-em-portugu%C3%AAs-utilizando-pytorch-e-python-91a232165ec0)
- Para realizar a busca de hiperprâmetros no modelo achei a documentação abaixo, mas não consegui utilizar porque há um bug na integração com o `Ray Tune`.
    - [Hyperparameter Search with Transformers and Ray Tune](https://huggingface.co/blog/ray-tune)
- O link abaixxo é um notebook de exemplo para realizar a classificação de texto utilizando apenas as ferramentas da HuggingFace. Quase tudo nesse notebook foi tirado daqui. O segundo link é o tutorial de como monitorar o treinamento com o TensorBoard.
    - [Text Classification on GLUE using `Trainer`](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb#scrollTo=8sgjdLKcIrJm)
    - [BERT Finetuning with Hugging Face and Training Visualizations with TensorBoard](https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97)
- Em algum momento percebi que o biblioteca da HuggingFace não calcula as métricas de performance do modelo no dataset de treino, impedindo a detecção de um possível overfitting. Para lidar com isso eu cheguei nos links abaixo:
    - [How to tweak `Trainer` to monitor other metrics on the training set](https://discuss.huggingface.co/t/metrics-for-training-set-in-trainer/2461/3)
    - [Batch and Epoch training metrics for transformers `Trainer`](https://stackoverflow.com/questions/78311534/batch-and-epoch-training-metrics-for-transformers-trainer/78311535#78311535)

- [Performance tips for training](https://huggingface.co/docs/transformers/v4.18.0/en/performance)

## Configurações

In [1]:
%%shell
pip install -q transformers datasets evaluate accelerate
pip install -q torch torchtext torchdata
pip install -q optuna
pip install -q ray[tune]
pip install -q hyperopt
pip install -q scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.9 MB/s eta 0:00:00


In [2]:
import os
import torch
import evaluate
import numpy as np

from copy import deepcopy

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    TrainerCallback
)

from datasets import (load_from_disk,
                      DatasetDict)

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [3]:
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/sentiment-analysis/')

Mounted at /content/drive


In [4]:
print(os.getcwd())

/content/drive/MyDrive/sentiment-analysis


In [5]:
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_LABELS = 3
MAX_LENGTH = 128
TASK = "sentiment-analysis"
MODEL_NAME = "bertimbau"

ID2LABEL = {0: "Neutro", 1: "Positivo", 2: "Negativo"}
LABEL2ID = {"Neutro": 0, "Positivo": 1, "Negativo": 2}
MODEL_CHECKPOINT = "neuralmind/bert-base-portuguese-cased"

OUTPUT_DIR = f"models/{MODEL_NAME}-finetuned-{TASK}"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT, num_labels=NUM_LABELS, id2label=ID2LABEL, label2id=LABEL2ID
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
from torch import nn
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
from transformers.trainer import _is_peft_model
from transformers.trainer import unwrap_model

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epoch_predctions = []
        self.epoch_labels = []
        self.epoch_loss = []

    def compute_loss(self, model, inputs, return_outputs=False):

        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        outputs = model(**inputs)

        print(type(outputs))
        print(type(inputs))
        #print(labels)
        #print(outputs)


        # metrics = self.compute_metrics(outputs)
        # self.log(metrics)

        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            unwrapped_model = unwrap_model(model)
            if _is_peft_model(unwrapped_model):
                model_name = unwrapped_model.base_model.model._get_name()
            else:
                model_name = unwrapped_model._get_name()
            if model_name in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss


class GetTrainingMetricsCallback(TrainerCallback):

    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(
                eval_dataset=self._trainer.train_dataset, metric_key_prefix="train"
            )
            return control_copy


def tokenize_function(examples: DatasetDict):
    return tokenizer(
        examples["text"],
        padding="max_length",
        max_length=MAX_LENGTH,
        truncation=True
    )


def compute_metrics(eval_pred):

    print(eval_pred)
    print(type(eval_pred))

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="macro"
    )

    return {"accuracy": accuracy,
            "f1": f1,
            "precision": precision,
            "recall": recall}

NameError: name 'Trainer' is not defined

In [8]:
ds = load_from_disk(f"/content/drive/MyDrive/sentiment-analysis/data/intermediate/without-emoticons")
ds = ds.map(tokenize_function, batched=True)
ds.set_format("torch")

ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 630481
    })
    dev: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 135103
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 135104
    })
})

In [9]:
small_train_dataset = ds["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000))

# Hyperparameter Search

---

In [ ]:
hp_space = {
        "per_device_train_batch_size": [8, 16, 32, 64, 128],
        "per_device_eval_batch_size": [8, 16, 32, 64, 128],
        "num_train_epochs": ([2, 3, 4, 5, 6]),
        "weight_decay": (0.0, 0.3),
        "learning_rate": (1e-5, 5e-5),
        "adam_epsilon": (1e-10, 1e-6),
        "warmup_ratio": (0.01, 0.03),
    }

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
      MODEL_CHECKPOINT, num_labels=NUM_LABELS, id2label=ID2LABEL, label2id=LABEL2ID
    )

def optuna_hp_space(trial):
    return {
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32, 64, 128]),
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [8, 16, 32, 64, 128]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 4, 5, 6]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.01, 0.03),
    }

def objective(metrics):
    return metrics["eval_f1"]

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    seed=SEED,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    logging_strategy="steps",
    save_total_limit=2,
    save_only_model=True,
    metric_for_best_model="f1",
    skip_memory_metrics=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["dev"],
    compute_metrics=compute_metrics,
)

trainer.add_callback(GetTrainingMetricsCallback(trainer))

best_run = trainer.hyperparameter_search(
    hp_space=optuna_hp_space,
    direction="maximize",
    backend="optuna",
    n_trials=20,
    )

## Fine-Tuning with Step Logging

---

In [16]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=SEED,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=500,
    save_total_limit=2,
    save_only_model=False,
    metric_for_best_model="f1",
    report_to="tensorboard",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# trainer.add_callback(GetTrainingMetricsCallback(trainer))

trainer.train()

trainer.save_model(f"{OUTPUT_DIR}-full-dataset-no-hyperopt")

<class 'transformers.modeling_outputs.SequenceClassifierOutput'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>


Step,Training Loss,Validation Loss


<class 'transformers.modeling_outputs.SequenceClassifierOutput'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 94.88 MiB is free. Process 63757 has 22.07 GiB memory in use. Of the allocated memory 21.64 GiB is allocated by PyTorch, and 202.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [17]:
!nvidia-smi

Wed May  1 23:31:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   65C    P0              32W /  72W |  22605MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
!kill 63757

/bin/bash: line 1: kill: (63757) - No such process


## Fine-Tuning with Epoch Logging

---

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=SEED,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    save_total_limit=2,
    save_only_model=True,
    metric_for_best_model="f1",
    report_to="tensorboard",
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["dev"],
    compute_metrics=compute_metrics,
)

trainer.add_callback(GetTrainingMetricsCallback(trainer))

trainer.train()

trainer.save_model(f"{OUTPUT_DIR}-full-dataset-no-hyperopt")

Epoch,Training Loss,Validation Loss
